In [1]:
##############################################
import sys,os
import numpy as np
import pylab as py
##############################################
import pylab as py
from  matplotlib import rc
import matplotlib.pyplot as plt
%matplotlib inline  
%config InlineBackend.figure_format = 'retina' 
##############################################
PATH='../python/'
sys.path.append(PATH)  # path of the JAMLIB python folder is located
from corelib import JAMLIB
##############################################

# Code for LHAPDF grids

## JAMLIB testing

In [2]:
def testjam(x,Q2):

  print 'alphaS     = ',jamFF.get_alphaS(Q2)
  print 'no. of posteriors = ',jamFF.npos
  print 'xFF(ipos=0) = ',jamFF.get_XF(0,'up',x,Q2)

  xgrid = jamFF.get_Xgrid()
  xmin = np.min(xgrid)
  xmax = np.max(xgrid)
  print 'Xmin,Xmax: ', xmin,xmax
  print xgrid 

  Q2grid = jamFF.get_Q2grid()
  Q2min = np.min(Q2grid)
  Q2max = np.max(Q2grid)
  print 'Q2min,Q2max: ', Q2min,Q2max
  print Q2grid

## Writes .info file

In [3]:
def writeinfo():

  H=[]
  H.append('SetDesc: '+desc)
  H.append('SetIndex: %i'%index)
  H.append('Authors: '+auth)
  H.append('Reference: '+ref)
  H.append('Format: lhagrid1')
  H.append('DataVersion: %i'%ver)

  H.append('Particle: %i'%particle)
  H.append('Flavors: '+str(flavors))
  H.append('OrderQCD: %i'%ordQCD)
  H.append('ForcePositive: 0')   # 0: do nothing 1: force to 0 if negative
  flavscheme = 'variable' # used for any variable flavor number scheme, massive or not
  H.append('FlavorScheme: '+flavscheme)
  H.append('NumFlavors: %i'%nfl)

  if errortype=='MC':
    nsets = nsamples+1   # number of posteriors + average
    errtype = 'replicas'  # LHAPDF-speak for MC method
  elif errortype=='HESSIAN': 
    nsets = 2*nsamples+2  # 2*eigenvalues + central fit
    errtype = 'hessian'
  H.append('NumMembers: %i'%nsets)
  H.append('ErrorType: '+errtype)
  H.append('ErrorConfLevel: %f'%cl)
  H.append('XMin: %e'%xmin)
  H.append('XMax: %e'%xmax)
  H.append('QMin: %e'%Qmin)
  H.append('QMax: %e'%Qmax)

  H.append('MZ: %f'%mZ)
  H.append('MUp: %f'%mu)
  H.append('MDown: %f'%md)
  H.append('MStrange: %f'%ms)
  H.append('MCharm: %f'%mc)
  H.append('MBottom: %f'%mb)
  H.append('MTop: %f'%mt)

  H.append('AlphaS_MZ: %f'%jamFF.get_alphaS(mZ**2))
  H.append('AlphaS_OrderQCD: %f'%ordQCD)
  H.append('AlphaS_Type: ipol')
  H.append('AlphaS_Qs: '+str(Rgrid.tolist()))
  alphas = []
  for Q in Rgrid:
    alphas.append(jamFF.get_alphaS(Q**2))
  H.append('AlphaS_Vals: '+str(alphas))
  H.append('AlphaS_Lambda4: %f'%jamFF.get_alphaS(mc**2))
  H.append('AlphaS_Lambda5: %f'%jamFF.get_alphaS(mb**2))

  for l in H: print l 

## Write a single .dat file

In [4]:
def writedat(member):
  
  # headers
  if member==0: 
    type='central'
  else:
    type='replica'
  H = []
  H.append('PdfType: '+type)
  H.append('Format: lhagrid1')
  H.append('---')  
  for l in H: print l
    
  # prints out x and Q grids  
  xvals = ["%10.4e" % x for x in ygrid]
  print ' '.join(xvals)
  Qvals = ["%10.4e" % Q for Q in Rgrid]
  print ' '.join(Qvals)
  flavs = ["%2i" % f for f in flavors]
  print ' '.join(flavs)
  
  # prints out the x*PDF(x) grid
  npart = len(partons)
    
  full_dist = []
  ix = 0
  iQ = 0
  il = 0
  for x in ygrid: 
    for Q in Rgrid:
      if member==0:  # Avreraged distribution
        FF = np.array([0 for n in range(npart)])
        for i in range(nsamples): 
          FF = FF + np.array([jamFF.get_XF(i,f,x,Q**2) for f in partons])
        FF = FF/nsamples
      else:   # member-th posterior 
        FF = [jamFF.get_XF(member-1,f,x,Q**2) for f in partons]
        #if Q < mb: FF[0] = FF[9] = 0
        #if Q < mc: FF[1] = FF[8] = 0
      if Q < mb: FF[4] = 0
      if Q < mc: FF[3] = 0
      #FF.append([x,Q])
      full_dist.append(FF)
  for l in full_dist: 
    out = ''
    for i in range(len(l)):
      out += '{:12.5e} '.format(l[i])
    print out

  print '---'  

 

## Writes all the .info and .dat files in one folder

In [5]:
def create_LHAPDF_grid(setname):
  
  print "Generating grids for "+setname
  
  start = os.getcwd()

  # Creates and navigates to the right folder
  if not os.path.exists(setname):
      os.makedirs(setname)
  os.chdir(setname)

  stdout = sys.stdout 
  
  # Writes the .info file
  f = open(setname+'.info','w')   
  sys.stdout = f    # <---------- very fragile solution, needs improvement
  writeinfo()
  f.close()
  sys.stdout=stdout

  for igrid in range(nsamples+1):
    gridname = '{}_{:04d}.dat'.format(setname,igrid)
    f = open(gridname,'w')
    sys.stdout = f   # <---------- very fragile solution, needs improvement
    writedat(igrid)
    f.close()
    sys.stdout=stdout
  
  # goes back to the initial folder
  os.chdir(start)



## log(1-z) grid generating functions

May not be needed, in the end, but I leave them here for now

In [6]:
def log_grid(x,a,d):    # logarithmic grid spacing function
  return np.log((1-x)**a+d)
  
def inv_log_grid(y,a,d):   # undoes the log grid 
  return 1-(np.exp(y)-d)**(1/a)

# GRID PRODUCTION

In [12]:
# Select JAM analysis
#jam_analysis = "../python/JAM16/FFpion"
jam_analysis = PATH+"JAM16/FFkaon"
jamFF=JAMLIB(jam_analysis)


# Select LHAPDF set name for given JAM hadron
# and set the user-level appropriate LHAPDF flags 
# (see "SET LEVEL" in https://lhapdf.hepforge.org/config.html )
#
# NOTE:
#  
# setname = name of the set as it should appear in LHAPDF
# Particle = the hadron to which the PDFs belong, or the FF fragments into, see
#   http://pdg.lbl.gov/2015/reviews/rpp2015-rev-monte-carlo-numbering.pdf
#   For protons, neutrons, use format ±n nr nL nq1 nq2 nq3 nJ
#   E.g, proton = 2212 (u; u; d; spin 1/2 --> nJ=2*s+1)
#   For nuclei, use LZZZAAAI, see point 14 in PDG standard:
#   L nonzero is for hypernuclei, I represent isomer levels with
#   I = 0 the ground state, and Z and A should be self-explanatory.
#   Examples: 2212=proton   211=pi+  321=K+
# desc = Description of teh contents of the file - free form
# auth = Authors of the fit / set
# ver = version number: -1=beta; 1,2,3, ... official version, and updated/debugged ones if needed
# index = internal LHAPDF number scheme. Should uniquely identify the set, but important only if
#         set is to be included in teh official repository. Ask Andy Buckley <andy.buckley@cern.ch>
#         or the LHAPDF developers <lhapdf@projects.hepforge.org> for detail.
#         Format: index = nnxxx
#         nn (or nnn): assigned by Andy [LHAPDF]; for example 16=JAM fits - this to be discussed with Andy
#         xxx (or xx): up to us --> I would suggest 000-002 for JAM15; 003-004 for JAM16 FF; and so on as needed

if jam_analysis==PATH+'JAM16/FFpion':
  setname = "JAM16FF_pi_test"
  particle = 221
  desc = "'JAMFF16 fragmentation functions of gluons and charge-conjugation-even quarks \
into pi+ mesons (identical to pi- by charge conjugation). Returns x*FF(x). \
Flavors 1-5: q+ == q+qbar with q=d,u,s,c,b; Flavor 21: gluon. \
mem=0 => average of posteriors; mem=1-200 => FF posteriors.'"
  auth = "'[JAM collaboration] Sato N, Ethier J J, Melnitchouk W, Hirai M, Kumano S, Accardi A'"
  ref = "'arXiv:1609.00899 - e-mail accardi@jlab.org / sato@jlab.org'"
  ver = -1   
  index = 16003
elif jam_analysis==PATH+'JAM16/FFkaon':
  setname = "JAM16FF_K_test"
  particle = 321
  desc = "'JAMFF16 fragmentation functions of gluons and charge-conjugation-even quarks \
into K+ mesons (identical to K- by charge conjugation). Returns x*FF(x). \
Flavors 1-5: q+ == q+qbar with q=d,u,s,c,b; Flavor 21: gluon. \
mem=0 => average of posteriors; mem=1-200 => FF posteriors.'"
  auth = "'[JAM collaboration] Sato N, Ethier J J, Melnitchouk W, Hirai M, Kumano S, Accardi A'"
  ref = "'arXiv:1609.00899 - e-mail accardi@jlab.org / sato@jlab.org'"
  ver = -1 
  index = 16004


########################################################## 
                                                           
     _   _    __  __ _     ___ ____                        
    | | / \  |  \/  | |   |_ _| __ )                       
 _  | |/ _ \ | |\/| | |    | ||  _ \                       
| |_| / ___ \| |  | | |___ | || |_) |                      
 \___/_/   \_\_|  |_|_____|___|____/                       
                                                           
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ 
                                                           
Authors:                                                   
Nobuo Sato         (Jefferson Lab)                         
Jake Ethier        (College of William and Mary)           
Wally Melnitchouk  (Jefferson Lab)                         
Alberto Accardi    (Hampton University and Jefferson Lab)  
########################################################## 
loading JAM16/FFkaon tables [100%]


In [ ]:
testjam(0.1,10.)

In [13]:
# COLLECTS INFO ON JAM FIT

# QCD setup
ordQCD =1  # 1: NLO
nfl = 5
#           Usual PDFs, with q/qbar separation
#           bb  cb  sb  ub  db  d  u  s  c  b  g
#flavors = [-5, -4, -3, -2, -1, 1, 2, 3, 4, 5, 21]
#          DIS/SIA-only analysis --> only q+=q+ + q- is fitted
#          d+ u+ s+ c+ b+  g
flavors = [1, 2, 3, 4, 5, 21]
partons = ['dp','up','sp','cp','bp','g']

# This is for JAM FF or PDF, for others, need to change
errortype = 'MC'      # MC = Monte Carlo method --> nsamples = no. of posteriors
nsamples = jamFF.npos # HESSIAN = Hessian --> nsamples = no. free parameters
cl = 68.3             # confidence level

# Collects info from JAM fits
xgrid = jamFF.get_Xgrid()
xmin = np.min(xgrid)
xmax = np.max(xgrid)
Qgrid = jamFF.get_Q2grid()**0.5
Qmin = np.min(Qgrid)
Qmax = np.max(Qgrid)
mu = 0.
md = 0.
ms = 0.
mc = jamFF.mc2**0.5
mb = jamFF.mb2**0.5
mt = jamFF.mt2**0.5
mZ = jamFF.mZ2**0.5

# Creates the momentum fraction grid (ygrid), and Q-scale grid (Rgrid)
# that can be different from JAM grids, in principle
  #a=0.7
  #d=100.
  #min,ymax = log_grid(xgrid[0],a,d),log_grid(xgrid[-1],a,d)
  #y=np.linspace(ymin,ymax,len(xgrid))
  #ygrid = inv_log_grid(y,a,d)
ygrid=xgrid

Rmin,Rmax=np.log(Qgrid[0]),np.log(Qgrid[-1])
nQ2=100
R = np.linspace(Rmin,Rmax,nQ2)
Rgrid = np.exp(R)


In [15]:
writeinfo()

SetDesc: 'JAMFF16 fragmentation functions of gluons and charge-conjugation-even quarks   into K+ mesons (identical to K- by charge conjugation). Returns x*FF(x).   Flavors 1-5: q+ == q+qbar with q=d,u,s,c,b; Flavor 21: gluon.   mem=0 => average of posteriors; mem=1-200 => FF posteriors.'
SetIndex: 16004
Authors: '[JAM collaboration] Sato N, Ethier J J, Melnitchouk W, Hirai M, Kumano S, Accardi A'
Reference: 'arXiv:1609.00899 - e-mail accardi@jlab.org / sato@jlab.org'
Format: lhagrid1
DataVersion: -1
Particle: 321
Flavors: [1, 2, 3, 4, 5, 21]
OrderQCD: 1
ForcePositive: 0
FlavorScheme: variable
NumFlavors: 5
NumMembers: 201
ErrorType: replicas
ErrorConfLevel: 68.300000
XMin: 5.000000e-02
XMax: 1.000000e+00
QMin: 1.000000e+00
QMax: 1.000000e+05
MZ: 91.187000
MUp: 0.000000
MDown: 0.000000
MStrange: 0.000000
MCharm: 1.430000
MBottom: 4.300000
MTop: 172.900000
AlphaS_MZ: 0.118000
AlphaS_OrderQCD: 1.000000
AlphaS_Type: ipol
AlphaS_Qs: [1.0, 1.1233240329780274, 1.2618568830660204, 1.4174741629

In [10]:
writedat(1)

PdfType: replica
Format: lhagrid1
---
5.0000e-02 5.9596e-02 6.9192e-02 7.8788e-02 8.8384e-02 9.7980e-02 1.0758e-01 1.1717e-01 1.2677e-01 1.3636e-01 1.4596e-01 1.5556e-01 1.6515e-01 1.7475e-01 1.8434e-01 1.9394e-01 2.0354e-01 2.1313e-01 2.2273e-01 2.3232e-01 2.4192e-01 2.5152e-01 2.6111e-01 2.7071e-01 2.8030e-01 2.8990e-01 2.9949e-01 3.0909e-01 3.1869e-01 3.2828e-01 3.3788e-01 3.4747e-01 3.5707e-01 3.6667e-01 3.7626e-01 3.8586e-01 3.9545e-01 4.0505e-01 4.1465e-01 4.2424e-01 4.3384e-01 4.4343e-01 4.5303e-01 4.6263e-01 4.7222e-01 4.8182e-01 4.9141e-01 5.0101e-01 5.1061e-01 5.2020e-01 5.2980e-01 5.3939e-01 5.4899e-01 5.5859e-01 5.6818e-01 5.7778e-01 5.8737e-01 5.9697e-01 6.0657e-01 6.1616e-01 6.2576e-01 6.3535e-01 6.4495e-01 6.5455e-01 6.6414e-01 6.7374e-01 6.8333e-01 6.9293e-01 7.0253e-01 7.1212e-01 7.2172e-01 7.3131e-01 7.4091e-01 7.5051e-01 7.6010e-01 7.6970e-01 7.7929e-01 7.8889e-01 7.9848e-01 8.0808e-01 8.1768e-01 8.2727e-01 8.3687e-01 8.4646e-01 8.5606e-01 8.6566e-01 8.7525e-01 8.848

In [14]:
# CREATE THE GRIDS

create_LHAPDF_grid(setname)

Generating grids for JAM16FF_K_test


In [ ]:
print xgrid